In [4]:
# http://blog.yhat.com/posts/decision-marking-under-uncertainty-2.html

import pulp

# The resource usage for each product (columns)- Nominal Values
a = [[2, 1], [1, 2]]

# Vector of deviations in resource consumptions
a_hat = [[1, 0.5], [0.8, 1.5]]

# Resources available
b = [6, 6]
Gamma = [0.0, 0.0]

# Keep results here
Result = []
x1 = []
x2 = []
for G1 in range(21):
    # Gamma 1 is G1/20 percent of the total possible value for gamma 1 which is 2
    Gamma[0] = (G1/20.0) * 2

    # Record the results for each Gamma 1
    Res_i = []
    x1_i = []
    x2_i = []

    for G2 in range(21):
        # Gamma 2 is being calculated as a percentage of total possible value for Gamma2
        Gamma[1] = (G2/20.0) * 2

        # Create maximization model
        Production = pulp.LpProblem('Production planning', pulp.LpMaximize)

        # Define our decision variables >= 0
        x = {}
        for j in range(2):
            x[j] = pulp.LpVariable('x[%s]' % j, 0)

        # Define y_j >= 0
        y = {}
        for j in range(2):
            y[j] = pulp.LpVariable('y[%s]' % j, 0)

        # Define z_i >= 0
        z = {}
        for i in range(2):
            z[i] = pulp.LpVariable('z[%s]' % i, 0)

        # Define p_ij >= 0
        p = {}
        for i in range(2):
            for j in range(2):
                p[i, j] = pulp.LpVariable('p[%s,%s]' % (i,j), 0)

        # Our objective is to maximize production
        Produced = pulp.LpVariable('Produced', 0)
        Production += Produced == sum(x[j] for j in range(2))
        Production.setObjective(Produced)

        # Define resource constraints
        for i in range(2):
            Production += sum(a[i][j] * x[j] for j in range(2)) + Gamma[i] * z[i] + sum(p[i, j] for j in range(2)) <= b[i]

        # Define each deviation for each uncertain parameter
        for i in range(2):
            for j in range(2):
                Production += z[i] + p[i, j] >= a_hat[i][j] * y[j]

        # Relate deviations to the decision variables
        for j in range(2):
            Production += x[j] <= y[j]

        # Solve the modelimport random
import math

b = [6, 6]

# We have 4 parameters that can deviate 1-a_11  2-a_12  3-a_21 4-a_22
a_nom = [2.0, 1.0, 1.0, 2.0]
a_dev = [1.0, 0.5, 0.8, 1.5]

x = [
    [2,      2],
    [1.963,  1.8777],
    [1.9212, 1.7734],
    [1.877,  1.6828],
    [1.8321, 1.6031],
    [1.7872, 1.5319],
    [1.7431, 1.4679],
    [1.7001, 1.4098],
    [1.6583, 1.3568],
    [1.6179, 1.3081],
    [1.5789, 1.2632],
    [1.5695, 1.23],
    [1.5611, 1.1969],
    [1.5539, 1.1638],
    [1.5478, 1.1306],
    [1.5429, 1.0971],
    [1.5392, 1.0634],
    [1.5368, 1.0293],
    [1.5358, 0.9946],
    [1.5363, 0.9593],
    [1.5385, 0.9231]
 ]

# This is a simulation for assessing the quality of the solution
Uni_Result = [] # Results for Uniform distribution
Tri_Result = [] # Results for Triangular distribution

# Lists to capture the risk magnitude for each resource and each distribution
Labor_Res = []
Lumber_Res = []

# Number of replications
Rep = 1000

#for each solution
for sol in range(21):
    Res_i = []
    triRes_i = []
    Labor = [[],[]]
    Lumber = [[],[]]

    # The different combinations that parameters can deviate.
    # 0 if no parameter deviates, ..., 4 if all 4 parameters deviate.
    # Note that when notheing goes wrong (j=0) we already know the result of the
    # simulation. We use it to calculate the probability of infeasibility,
    # If any of the 2 consrtaints become infeasible we have infeasibility in the solution.

    for i in range(5):
        # Counters for the simulation of each level (levels are the number of parameters deviating)
        Counter = 0.0
        CounterTri = 0.0

        # Counters to count the number of times each specific constraint is violated
        CounterU = [0.0, 0.0]   # how many times there was infeaibility for Uniform Distribution 
        CounterT = [0.0, 0.0]   # how many times there was infeaibility for Tiangular Distribution 
        LaborOverU = 0          # Total overage(infeasibility) in labor (only if it is infeasible) for Uniform
        LumberOverU = 0         # Total overage(infeasibility) in lumber(only if it is infeasible) for Uniform
        LaborOverT = 0          # Total overage(infeasibility) in labor (only if it is infeasible) for Triangular
        LumberOverT = 0         # Total overage(infeasibility) in lumber (only if it is infeasible) for Triangular

        for j in range(Rep):
            A = range(4)

            # List of elements that deviate from their nominal value
            # Initiate a list with 4 False values for none of them to deviate
            DevList = [False] * 4

            # Initialize the value for the realization of parameter a
            a_realized = []     # Uniform distribution
            a_realTri = []      # Triangular distribution

            # If i == 0 it will not enter the loop and no values will change to True
            for k in range(i):
                # Choose an index to deviate
                # Add it to the list of deviations
                # Remove it from the original List of all indexes
                DevItem = random.choice(A)
                DevList[DevItem] = True
                A.remove(DevItem)

            for h in range(4):
                if DevList[h] == True:
                    # If there has to be a deviation, we generate a random number from the uniform
                    # istribution starting in a_nom plus/minus a_hat
                    L = a_nom[h] - a_dev[h]
                    U = a_nom[h] + a_dev[h]
                    aRand = round(random.uniform(L, U), 4)
                    aRandTri = round(random.triangular(L, U, a_nom[h]), 4)
                    a_realized.append(aRand)
                    a_realTri.append(aRandTri)

                else:
                    a_realized.append(a_nom[h])
                    a_realTri.append(a_nom[h])

            # Now we need to check to see if the constraints are met!
            # Compute the excess of resources for the realization of parameter a and decision x 
            # with Gamma level at (sol/20)*2 
            s1 = b[0] - ((a_realized[0] * x[sol][0]) + (a_realized[1] * x[sol][1]))
            s2 = b[1] - ((a_realized[2] * x[sol][0]) + (a_realized[3] * x[sol][1]))

            # If one of the slacks are negative we have infeasibility
            if (s1 < 0 or s2 < 0):
                Counter += 1

            # Check the constraint violation and its magnitude (Uniform)- Labor
            if s1 < 0:
                CounterU[0] += 1
                LaborOverU += abs(s1)

            # Check the constraint violation and its magnitude (Uniform)- Lumber
            if s2 < 0:
                CounterU[1] += 1
                LumberOverU += abs(s2)

            # Check the results with triangular distribution
            s1 = b[0] - ((a_realTri[0] * x[sol][0]) + (a_realTri[1] * x[sol][1]))
            s2 = b[1] - ((a_realTri[2] * x[sol][0]) + (a_realTri[3] * x[sol][1]))

            # If one of the slacks is negative we have infeasibility
            if (s1 < 0 or s2 < 0):
                CounterTri += 1

            # Check the constraint violation and its magnitude (Triangular)- Labor
            if s1 < 0:
                CounterT[0] += 1
                LaborOverT += abs(s1)

            # Checking the constraint violation and its magnitude (Triangular)- Lumber
            if s2 < 0:
                CounterT[1] += 1
                LumberOverT += abs(s2)

        # Labor
        if CounterU[0] != 0:
            AvgLaborU = LaborOverU/CounterU[0]
        else:
            AvgLaborU = 0
        Labor[0].append(round(AvgLaborU, 4))

        if CounterT[0] != 0:    
            AvgLaborT = LaborOverT/CounterT[0]
        else:
            AvgLaborT = 0
        Labor[1].append(round(AvgLaborT, 4))

        # Lumber
        if CounterU[1] != 0 :
            AvgLumberU = LumberOverU/CounterU[1]
        else:
            AvgLumberU = 0
        Lumber[0].append(round(AvgLumberU, 4))

        if CounterT[1] != 0:
            AvgLumberT = LumberOverT/CounterT[1]
        else:
            AvgLumberT = 0
        Lumber[1].append(round(AvgLumberT, 4))

        # Uniform
        Avg_Infeasvility_Probability = Counter/Rep
        Res_i.append(Avg_Infeasvility_Probability)

        # Triangular
        Avg_Infeasvility_Probability = CounterTri/Rep
        triRes_i.append(Avg_Infeasvility_Probability)

    Uni_Result.append(Res_i)
    Tri_Result.append(triRes_i)
    Labor_Res.append(Labor)
    Lumber_Res.append(Lumber)

print "Probability of Infeasibility : Uniform Distribution || Triangular DIstribution  "
for i in range(21):
    print "Gamma =", (i/20.0) * 2,
    print Uni_Result[i], Tri_Result[i]
print

print "Labor Results (Uniform || Triangular): "
for i in range(21):
    print "Gamma =", (i/20.0) * 2,
    print Labor_Res[i]
print

print "Lumber Results (Uniform || Triangular): "
for i in range(21):
    print "Gamma =", (i/20.0) * 2,
    print Lumber_Res[i]
        status = Production.solve()
        assert status == pulp.LpStatusOptimal

        Res_i.append(pulp.value(Produced))

        # Record optimal plans
        x1_i.append(pulp.value(x[0]))
        x2_i.append(pulp.value(x[1]))

    # Save results
    Result.append(Res_i)
    x1.append(x1_i)
    x2.append(x2_i)

print('G1=G2\tx1\tx2')
for i in range(21):
    print('%.1f\t%.4f\t%.4f' % ((i/20.0) * 2, x1[i][i], x2[i][i]))

/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1137: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


G1=G2	x1	x2
0.0	2.0000	2.0000
0.1	1.9630	1.8777
0.2	1.9212	1.7734
0.3	1.8770	1.6828
0.4	1.8321	1.6031
0.5	1.7872	1.5319
0.6	1.7431	1.4679
0.7	1.7001	1.4098
0.8	1.6583	1.3568
0.9	1.6179	1.3081
1.0	1.5789	1.2632
1.1	1.5695	1.2300
1.2	1.5611	1.1969
1.3	1.5539	1.1638
1.4	1.5478	1.1306
1.5	1.5429	1.0971
1.6	1.5392	1.0634
1.7	1.5368	1.0293
1.8	1.5358	0.9946
1.9	1.5363	0.9593
2.0	1.5385	0.9231


In [5]:
import random
import math

b = [6, 6]

# We have 4 parameters that can deviate 1-a_11  2-a_12  3-a_21 4-a_22
a_nom = [2.0, 1.0, 1.0, 2.0]
a_dev = [1.0, 0.5, 0.8, 1.5]

x = [
    [2,      2],
    [1.963,  1.8777],
    [1.9212, 1.7734],
    [1.877,  1.6828],
    [1.8321, 1.6031],
    [1.7872, 1.5319],
    [1.7431, 1.4679],
    [1.7001, 1.4098],
    [1.6583, 1.3568],
    [1.6179, 1.3081],
    [1.5789, 1.2632],
    [1.5695, 1.23],
    [1.5611, 1.1969],
    [1.5539, 1.1638],
    [1.5478, 1.1306],
    [1.5429, 1.0971],
    [1.5392, 1.0634],
    [1.5368, 1.0293],
    [1.5358, 0.9946],
    [1.5363, 0.9593],
    [1.5385, 0.9231]
 ]

# This is a simulation for assessing the quality of the solution
Uni_Result = [] # Results for Uniform distribution
Tri_Result = [] # Results for Triangular distribution

# Lists to capture the risk magnitude for each resource and each distribution
Labor_Res = []
Lumber_Res = []

# Number of replications
Rep = 1000

#for each solution
for sol in range(21):
    Res_i = []
    triRes_i = []
    Labor = [[],[]]
    Lumber = [[],[]]

    # The different combinations that parameters can deviate.
    # 0 if no parameter deviates, ..., 4 if all 4 parameters deviate.
    # Note that when notheing goes wrong (j=0) we already know the result of the
    # simulation. We use it to calculate the probability of infeasibility,
    # If any of the 2 consrtaints become infeasible we have infeasibility in the solution.

    for i in range(5):
        # Counters for the simulation of each level (levels are the number of parameters deviating)
        Counter = 0.0
        CounterTri = 0.0

        # Counters to count the number of times each specific constraint is violated
        CounterU = [0.0, 0.0]   # how many times there was infeaibility for Uniform Distribution 
        CounterT = [0.0, 0.0]   # how many times there was infeaibility for Tiangular Distribution 
        LaborOverU = 0          # Total overage(infeasibility) in labor (only if it is infeasible) for Uniform
        LumberOverU = 0         # Total overage(infeasibility) in lumber(only if it is infeasible) for Uniform
        LaborOverT = 0          # Total overage(infeasibility) in labor (only if it is infeasible) for Triangular
        LumberOverT = 0         # Total overage(infeasibility) in lumber (only if it is infeasible) for Triangular

        for j in range(Rep):
            A = range(4)

            # List of elements that deviate from their nominal value
            # Initiate a list with 4 False values for none of them to deviate
            DevList = [False] * 4

            # Initialize the value for the realization of parameter a
            a_realized = []     # Uniform distribution
            a_realTri = []      # Triangular distribution

            # If i == 0 it will not enter the loop and no values will change to True
            for k in range(i):
                # Choose an index to deviate
                # Add it to the list of deviations
                # Remove it from the original List of all indexes
                DevItem = random.choice(A)
                DevList[DevItem] = True
                A.remove(DevItem)

            for h in range(4):
                if DevList[h] == True:
                    # If there has to be a deviation, we generate a random number from the uniform
                    # istribution starting in a_nom plus/minus a_hat
                    L = a_nom[h] - a_dev[h]
                    U = a_nom[h] + a_dev[h]
                    aRand = round(random.uniform(L, U), 4)
                    aRandTri = round(random.triangular(L, U, a_nom[h]), 4)
                    a_realized.append(aRand)
                    a_realTri.append(aRandTri)

                else:
                    a_realized.append(a_nom[h])
                    a_realTri.append(a_nom[h])

            # Now we need to check to see if the constraints are met!
            # Compute the excess of resources for the realization of parameter a and decision x 
            # with Gamma level at (sol/20)*2 
            s1 = b[0] - ((a_realized[0] * x[sol][0]) + (a_realized[1] * x[sol][1]))
            s2 = b[1] - ((a_realized[2] * x[sol][0]) + (a_realized[3] * x[sol][1]))

            # If one of the slacks are negative we have infeasibility
            if (s1 < 0 or s2 < 0):
                Counter += 1

            # Check the constraint violation and its magnitude (Uniform)- Labor
            if s1 < 0:
                CounterU[0] += 1
                LaborOverU += abs(s1)

            # Check the constraint violation and its magnitude (Uniform)- Lumber
            if s2 < 0:
                CounterU[1] += 1
                LumberOverU += abs(s2)

            # Check the results with triangular distribution
            s1 = b[0] - ((a_realTri[0] * x[sol][0]) + (a_realTri[1] * x[sol][1]))
            s2 = b[1] - ((a_realTri[2] * x[sol][0]) + (a_realTri[3] * x[sol][1]))

            # If one of the slacks is negative we have infeasibility
            if (s1 < 0 or s2 < 0):
                CounterTri += 1

            # Check the constraint violation and its magnitude (Triangular)- Labor
            if s1 < 0:
                CounterT[0] += 1
                LaborOverT += abs(s1)

            # Checking the constraint violation and its magnitude (Triangular)- Lumber
            if s2 < 0:
                CounterT[1] += 1
                LumberOverT += abs(s2)

        # Labor
        if CounterU[0] != 0:
            AvgLaborU = LaborOverU/CounterU[0]
        else:
            AvgLaborU = 0
        Labor[0].append(round(AvgLaborU, 4))

        if CounterT[0] != 0:    
            AvgLaborT = LaborOverT/CounterT[0]
        else:
            AvgLaborT = 0
        Labor[1].append(round(AvgLaborT, 4))

        # Lumber
        if CounterU[1] != 0 :
            AvgLumberU = LumberOverU/CounterU[1]
        else:
            AvgLumberU = 0
        Lumber[0].append(round(AvgLumberU, 4))

        if CounterT[1] != 0:
            AvgLumberT = LumberOverT/CounterT[1]
        else:
            AvgLumberT = 0
        Lumber[1].append(round(AvgLumberT, 4))

        # Uniform
        Avg_Infeasvility_Probability = Counter/Rep
        Res_i.append(Avg_Infeasvility_Probability)

        # Triangular
        Avg_Infeasvility_Probability = CounterTri/Rep
        triRes_i.append(Avg_Infeasvility_Probability)

    Uni_Result.append(Res_i)
    Tri_Result.append(triRes_i)
    Labor_Res.append(Labor)
    Lumber_Res.append(Lumber)

print("Probability of Infeasibility : Uniform Distribution || Triangular DIstribution  ")
for i in range(21):
    print("Gamma =", (i/20.0) * 2)
    print(Uni_Result[i], Tri_Result[i])
    
print("")

print("Labor Results (Uniform || Triangular): ")
for i in range(21):
    print("Gamma =", (i/20.0) * 2)
    print(Labor_Res[i])

print("")

print("Lumber Results (Uniform || Triangular): ")
for i in range(21):
    print("Gamma =", (i/20.0) * 2)
    print(Lumber_Res[i])

AttributeError: 'range' object has no attribute 'remove'